O objetivo desse notebook é construir uma base de utilidade para cada aeroporto nacional para o ano de 2024. A motivação para isso parte da literatura de competitividade, em destaque o Huff Model, que constroe um modelo
de decisão do consumidor baseada nas diferentes características disponíveis dos ofertantes de determinado produto ou serviço. Este modelo é construído com a premissa que a escolha do consumidor é baseada no leque de ofertantes (e suas respectivas utilidades) e na distância geográfica desse consumidor para os ofertantes em questão.

A aplicação dessa metodologia em análise de aeroportos foi feita em Teixeira et al (2021)¹, que utilizou o modelo Huff para calcular a atratividade dos aeroportos da região próxima à Nova York para cada bairro da região  metropolitana disponível pelo censo americano. Os autores construiram um modelo Huff baseado em três vetores de utilidade (i) utilidade da tarifa; (ii) utilidade da conectividade; e (iii) utilidade da pontualidade. Esses três vetores que são adaptados para a realidade nacional nesse presente notebook. A seguir são apresentados cada um dos vetores: 

1. **Utilidade de Tarifa**  
   $
   ut_i = \sum \text{tarifa\_normalizada\_por\_destino}_i
   $

2. **Utilidade de Conectividade** 
   $
   uc_i = \text{destino\_totais}_i + \text{decolagens\_totais}_i + \text{destinos\_diretos}_i + \text{destinos\_unicos}_i
   $


3. **Utilidade de Pontualidade**  
   $
   up_i = \text{cancelados}_i + \text{atrasados}_i + \text{pontuais}_i
   $


4. **Utilidade Geral**  
   $
   U_i = ut_i + uc_i + up_i
   $

**Todos os indicadores individuas (destinos, decolagens...) devem ser normalizados e o indicador final também**

¹Teixeira, Filipe Marques, and Ben Derudder. "Spatio-temporal dynamics in airport catchment areas: The case of the New York Multi Airport Region." Journal of Transport Geography 90 (2021): 102916

In [26]:
import pandas as pd
import os
import requests
from io import StringIO
import warnings

In [112]:
#Buscando a base dados estatísticos no sita da ANAC
#Essa base contem informações diversas do setor: n° de pax, carga, distancia, decolagens, etc.
url = "https://sistemas.anac.gov.br/dadosabertos/Voos%20e%20opera%C3%A7%C3%B5es%20a%C3%A9reas/Dados%20Estat%C3%ADsticos%20do%20Transporte%20A%C3%A9reo/Dados_Estatisticos.csv"
dados_estatisticos = pd.read_csv(url, encoding='utf-8', delimiter=';', skiprows=1) 

In [127]:
#Filtrando somente os dados de 2024
base_aerea = dados_estatisticos[dados_estatisticos["ANO"] == 2024]
base_aerea.columns

Index(['EMPRESA_SIGLA', 'EMPRESA_NOME', 'EMPRESA_NACIONALIDADE', 'ANO', 'MES',
       'AEROPORTO_DE_ORIGEM_SIGLA', 'AEROPORTO_DE_ORIGEM_NOME',
       'AEROPORTO_DE_ORIGEM_UF', 'AEROPORTO_DE_ORIGEM_REGIAO',
       'AEROPORTO_DE_ORIGEM_PAIS', 'AEROPORTO_DE_ORIGEM_CONTINENTE',
       'AEROPORTO_DE_DESTINO_SIGLA', 'AEROPORTO_DE_DESTINO_NOME',
       'AEROPORTO_DE_DESTINO_UF', 'AEROPORTO_DE_DESTINO_REGIAO',
       'AEROPORTO_DE_DESTINO_PAIS', 'AEROPORTO_DE_DESTINO_CONTINENTE',
       'NATUREZA', 'GRUPO_DE_VOO', 'PASSAGEIROS_PAGOS', 'PASSAGEIROS_GRATIS',
       'CARGA_PAGA_KG', 'CARGA_GRATIS_KG', 'CORREIO_KG', 'ASK', 'RPK', 'ATK',
       'RTK', 'COMBUSTIVEL_LITROS', 'DISTANCIA_VOADA_KM', 'DECOLAGENS',
       'CARGA_PAGA_KM', 'CARGA_GRATIS_KM', 'CORREIO_KM', 'ASSENTOS', 'PAYLOAD',
       'HORAS_VOADAS', 'BAGAGEM_KG'],
      dtype='object')

In [128]:
base_aerea_filtrada = base_aerea[base_aerea['AEROPORTO_DE_ORIGEM_PAIS'] == 'BRASIL']

#Criação da variável de quantidade de decolagens por aeroporto
decolagens = pd.DataFrame(base_aerea_filtrada.groupby('AEROPORTO_DE_ORIGEM_SIGLA')['DECOLAGENS'].sum())
decolagens = decolagens.rename(columns={'DECOLAGENS': "Decolagens"})

#Variável de qunatidade de voos diretos
mercados_diretos = pd.DataFrame(base_aerea_filtrada.groupby('AEROPORTO_DE_ORIGEM_SIGLA')['AEROPORTO_DE_DESTINO_SIGLA'].count())


Tentativa fazer o download das planilhas de tarifa diretamente do site da ANAC

In [ ]:
pip install selenium beautifulsoup4


In [42]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options  # Importar a classe Options corretamente
import time

In [ ]:
chrome_options = Options()  
chrome_options.add_argument("--headless") 
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

chrome_driver_path = "C:/Users/GUilherme.costa/Downloads/chromedriver-win32/chromedriver-win32/chromedriver.exe"
               
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)


try:
    driver.get("https://sas.anac.gov.br/sas/downloads/view/frmDownload.aspx?tema=14")
    
    time.sleep(5)
    
    select_ano = Select(driver.find_element(By.ID, "MainContet_listAno"))
    select_ano.select_by_visible_text("2024")
    
    select_tema = Select(driver.find_element(By.ID, "MainContent_listTema"))  
    select_tema.select_by_visible_text("Tarifas Transporte Aéreo Passagerios Domésticos")
    
    botao_buscar = driver.find_element(By.ID, "MainContent_btnListaArquivos")  
    botao_buscar.click()
    
    time.sleep(5)
    
    botao_selecionartodos = driver.find_element(By.ID, "MainContent_btnMarcar")
    botao_selecionartodos.click()

    botao_download = driver.find_element(By.ID, "MainContent_btnBaixar")
    botao_download.click()
    
    time.sleep(10)  

finally:
    driver.quit()


In [131]:
#O código acima ainda está dando erro. Acredito que estou com problema para acessar no site da Anac. Ainda pretendo voltar aqui para automatizar
#Enquanto isso, vou fazer download dos arquivos na mão, como um boomer

caminho_pasta = "C:/Users/GUilherme.costa/Downloads/anac_14925995"
arquivos_csv = [arquivo for arquivo in os.listdir(caminho_pasta) if arquivo.lower().endswith('.csv')]

dataframes = []

for arquivo in arquivos_csv:
    caminho_arquivo = os.path.join(caminho_pasta, arquivo)
    df = pd.read_csv(caminho_arquivo, encoding='utf-8', delimiter=';') 
    dataframes.append(df)

base_tarifa = pd.concat(dataframes, ignore_index=True)

base_tarifa['TARIFA'] = (
    df['TARIFA']
    .str.replace('.', '', regex=False)  
    .str.replace(',', '.')              
    .apply(pd.to_numeric, errors='coerce') 
)


In [132]:
#Para o cálculo da tarifa não da pra fazer apenas uma média, é preciso ponderar por  assento por rota. Vou fazer isso para o ano
base_tarifa['OD'] = base_tarifa['ORIGEM'] +'-' + base_tarifa['DESTINO']
base_tarifa['TARIFA_ASSENTO'] = base_tarifa['TARIFA']*base_tarifa['ASSENTOS']  

#Cálculo do total de assentos por roda
soma_assentos_por_rota = base_tarifa.groupby('OD')['ASSENTOS'].sum().reset_index()
soma_assentos_por_rota.columns = ['OD', 'SOMA_ASSENTOS']

#Cálculo da soma das tarifas por rota
soma_tarifa_assentos = base_tarifa.groupby('OD')['TARIFA_ASSENTO'].sum().reset_index()
soma_tarifa_assentos.columns = ['OD', 'SOMA_TARIFA*ASSENTOS']

tarifa_ponderada = soma_assentos_por_rota.merge(soma_tarifa_assentos, on = 'OD' )
tarifa_ponderada['Tarifa_P'] = tarifa_ponderada['SOMA_TARIFA*ASSENTOS'] / tarifa_ponderada['SOMA_ASSENTOS']
tarifa_ponderada = tarifa_ponderada.drop(['SOMA_ASSENTOS',	'SOMA_TARIFA*ASSENTOS'], axis =1)

In [133]:
#Juntando as duas bases. Reparar que a tarifa ponderada é a mesma para o mesmo OD naquele ano
base_tarifa = base_tarifa.merge(tarifa_ponderada, on='OD')
base_tarifa.head()

,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA,ASSENTOS,OD,TARIFA_ASSENTO,Tarifa_P
0,2024,1,ABJ,SBSV,SDLO,750.0,8,SBSV-SDLO,6000.0,181.224839
1,2024,1,ABJ,SBSV,SDLO,950.0,8,SBSV-SDLO,7600.0,181.224839
2,2024,1,ABJ,SBSV,SDLO,1150.0,14,SBSV-SDLO,16100.0,181.224839
3,2024,1,ABJ,SBSV,SDLO,600.0,1,SBSV-SDLO,600.0,181.224839
4,2024,1,ABJ,SBSV,SIRI,700.0,11,SBSV-SIRI,7700.0,102.516109


In [134]:
base_aeros = base_tarifa

#Incluir destinos nessa base
destinos = base_aeros.groupby(['ORIGEM', 'OD']).size()
destinos = pd.DataFrame(destinos.groupby('ORIGEM').size())
base_aeros = pd.merge(base_aeros, destinos, on='ORIGEM', how='left')
base_aeros = base_aeros.rename(columns={0: "n_Destinos"})
base_aeros.head()

,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA,ASSENTOS,OD,TARIFA_ASSENTO,Tarifa_P,n_Destinos
0,2024,1,ABJ,SBSV,SDLO,750.0,8,SBSV-SDLO,6000.0,181.224839,135
1,2024,1,ABJ,SBSV,SDLO,950.0,8,SBSV-SDLO,7600.0,181.224839,135
2,2024,1,ABJ,SBSV,SDLO,1150.0,14,SBSV-SDLO,16100.0,181.224839,135
3,2024,1,ABJ,SBSV,SDLO,600.0,1,SBSV-SDLO,600.0,181.224839,135
4,2024,1,ABJ,SBSV,SIRI,700.0,11,SBSV-SIRI,7700.0,102.516109,135


In [135]:
#Incluir decolagens 
base_aeros = pd.merge(base_aeros, decolagens, left_on="ORIGEM", right_on="AEROPORTO_DE_ORIGEM_SIGLA", how="left" )
base_aeros.head()

,ANO,MES,EMPRESA,ORIGEM,DESTINO,TARIFA,ASSENTOS,OD,TARIFA_ASSENTO,Tarifa_P,n_Destinos,Decolagens
0,2024,1,ABJ,SBSV,SDLO,750.0,8,SBSV-SDLO,6000.0,181.224839,135,27645.0
1,2024,1,ABJ,SBSV,SDLO,950.0,8,SBSV-SDLO,7600.0,181.224839,135,27645.0
2,2024,1,ABJ,SBSV,SDLO,1150.0,14,SBSV-SDLO,16100.0,181.224839,135,27645.0
3,2024,1,ABJ,SBSV,SDLO,600.0,1,SBSV-SDLO,600.0,181.224839,135,27645.0
4,2024,1,ABJ,SBSV,SIRI,700.0,11,SBSV-SIRI,7700.0,102.516109,135,27645.0


In [137]:
#Pegar informações sobre atrasos e cancelamentos

warnings.filterwarnings('ignore', message='Unverified HTTPS request')

def download_anac():
    url_patterns = [
        "https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2024/vra_2024_{:02d}.csv",
        "https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/percentuais-de-atrasos-e-cancelamentos-2/2024/VRA_2024_{:02d}.csv"
    ]
    
    dfs = []  
    meses_baixados = []  
    
    for month in range(1, 13):
        downloaded = False
        for url_pattern in url_patterns:
            url = url_pattern.format(month)
            try:
                response = requests.get(url, verify=False, timeout=30)
                response.raise_for_status()  
                
                df = pd.read_csv(StringIO(response.text), sep=';', encoding='utf-8')
                df['Mês'] = month  
                dfs.append(df)
                meses_baixados.append(month)
                downloaded = True
                
                print(f"Arquivo do mês {month:02d} baixado com sucesso (formato: {'maiúsculo' if 'VRA' in url else 'minúsculo'}). Linhas: {len(df)}")
                break  
                
            except requests.exceptions.HTTPError:
                continue  
            except Exception as e:
                print(f"Erro ao processar mês {month:02d} (URL: {url}): {str(e)}")
                continue
        
        if not downloaded:
            print(f"Arquivo do mês {month:02d} não disponível em nenhum formato. Ignorando...")
    
    if dfs:
        final_df = pd.concat(dfs, ignore_index=True)
        print(f"\nProcesso concluído. Total de meses baixados: {len(set(meses_baixados))}")
        print(f"DataFrame final com {len(final_df)} linhas.")
        return final_df
    else:
        print("Nenhum arquivo foi baixado. Verifique a disponibilidade dos dados.")
        return pd.DataFrame()

print("Iniciando download dos dados da ANAC...")
base_pontualidade = download_anac()



Iniciando download dos dados da ANAC...
Arquivo do mês 01 baixado com sucesso (formato: minúsculo). Linhas: 86931
Arquivo do mês 02 baixado com sucesso (formato: maiúsculo). Linhas: 77484
Arquivo do mês 03 baixado com sucesso (formato: maiúsculo). Linhas: 81560
Arquivo do mês 04 baixado com sucesso (formato: minúsculo). Linhas: 80750
Arquivo do mês 05 baixado com sucesso (formato: minúsculo). Linhas: 78883
Arquivo do mês 06 baixado com sucesso (formato: maiúsculo). Linhas: 78078
Arquivo do mês 07 baixado com sucesso (formato: minúsculo). Linhas: 87814
Arquivo do mês 08 baixado com sucesso (formato: minúsculo). Linhas: 85385
Arquivo do mês 09 baixado com sucesso (formato: minúsculo). Linhas: 81479
Arquivo do mês 10 baixado com sucesso (formato: maiúsculo). Linhas: 84355
Arquivo do mês 11 não disponível em nenhum formato. Ignorando...
Arquivo do mês 12 não disponível em nenhum formato. Ignorando...

Processo concluído. Total de meses baixados: 10
DataFrame final com 822719 linhas.


In [138]:
#Calcular o % de voos atrasados, pontuais e cancelados por aeroporto em 2024
base_pontualidade = base_pontualidade[['Sigla ICAO Aeroporto Origem', 'Partida Prevista', 'Partida Real', 'SituaÃ§Ã£o Voo']]
base_pontualidade = base_pontualidade.rename(columns={'Sigla ICAO Aeroporto Origem':'Aero', 'SituaÃ§Ã£o Voo':'Situação Voo'})
base_pontualidade['Partida Prevista'] = pd.to_datetime(base_pontualidade['Partida Prevista'], format='%d/%m/%Y %H:%M')
base_pontualidade['Partida Real'] = pd.to_datetime(base_pontualidade['Partida Real'], format='%d/%m/%Y %H:%M',  errors='coerce')
base_pontualidade['Atraso'] = (base_pontualidade['Partida Real'] - base_pontualidade['Partida Prevista']).dt.total_seconds() / 60
base_pontualidade['Status'] = ['Atrasado' if atraso > 10 else 'Pontual' for atraso in base_pontualidade['Atraso']]

status_voo = base_pontualidade.groupby('Aero').agg(
    Total_Voos=('Aero', 'count'),
    Voos_Atrasados=('Status', lambda x: (x == 'Atrasado').sum()),
    Voos_Cancelados=('Situação Voo', lambda x: (x == 'CANCELADO').sum())
).reset_index()

status_voo['%_Atrasados'] = (status_voo['Voos_Atrasados'] / status_voo['Total_Voos']) * 100
status_voo['%_Cancelados'] = (status_voo['Voos_Cancelados'] / status_voo['Total_Voos']) * 100

status_voo['%_Atrasados'] = status_voo['%_Atrasados'].round(2)
status_voo['%_Cancelados'] = status_voo['%_Cancelados'].round(2)

status_voo['%_Pontuais'] = 100 - (status_voo['%_Cancelados']+status_voo['%_Atrasados'])

status_voo = status_voo.drop(['Total_Voos', 'Voos_Atrasados', 'Voos_Cancelados'], axis=1)


In [139]:
base_aeros = pd.merge(base_aeros, status_voo, left_on='ORIGEM', right_on='Aero', how='left')
base_final = base_aeros[['Aero', 'Tarifa_P', 'n_Destinos', 'Decolagens', '%_Atrasados', '%_Cancelados', '%_Pontuais', 'OD']]
base_final.head()

,Aero,Tarifa_P,n_Destinos,Decolagens,%_Atrasados,%_Cancelados,%_Pontuais,OD
0,SBSV,181.224839,135,27645.0,16.01,2.31,81.68,SBSV-SDLO
1,SBSV,181.224839,135,27645.0,16.01,2.31,81.68,SBSV-SDLO
2,SBSV,181.224839,135,27645.0,16.01,2.31,81.68,SBSV-SDLO
3,SBSV,181.224839,135,27645.0,16.01,2.31,81.68,SBSV-SDLO
4,SBSV,102.516109,135,27645.0,16.01,2.31,81.68,SBSV-SIRI


In [142]:
base_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7717161 entries, 0 to 7717160
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Aero          object 
 1   Tarifa_P      float64
 2   n_Destinos    int64  
 3   Decolagens    float64
 4   %_Atrasados   float64
 5   %_Cancelados  float64
 6   %_Pontuais    float64
 7   OD            object 
dtypes: float64(5), int64(1), object(2)
memory usage: 471.0+ MB


A 'base_final' é a base de utilidade dos aeroportos do Brasil. A partir dessa base é possível realizar análises de competição entre aeroportos de uma mesma região de influencia. Por exemplo, é possível calcular a utilidade de todos os aeroportos da região metropolitana de Sâo Paulo (SBGR, SBSP e SBKP) e assim verificar qual tem maior utilidade para os usuários. Lembrar q

Isso é util se combinado com a distancia de cada usuário (que eu vou explicar em outro notebook como conseguir) e, dessa maneira, calcular a atratividade de cada aeroporto para casa bairro, ou cidade, ou usuário. 

A base não está finalizada por ainda falta duas variáveis de utilidade que devem ser calculadas para cada região de influência: 
        1. tarifa: a tarifa foi calculada por OD, então o aeroporto que tiver a menor tarifa para determinado destino deverá ter a maior utilidade. No exemplo de SP, se for mais barato ir para Salvador por GRU esse aeroporto recebera valor 1 e o mais caro recebera valor 0 na normalização. Assim, soma-se as utilidades das rotas para se obter a utilidade final de tarifa. O aeroporto que tiver mais destino por preço mais baixo, é o que tem a utilidade de tarifa melhor naquela região de influencia. 
        2. Destinos unicos: somente faz sentido falarmos em destinos unicos quando olhar novamente para regiões de influencia, o aeroporto de Manaus não é um competidor direto do Aeroporto de Guarulhos. 

